In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize,PunktSentenceTokenizer
import matplotlib as mp
from nltk.corpus import state_union
import random
from nltk.corpus import movie_reviews
import pickle
from nltk.classify import ClassifierI
from statistics import mode
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from nltk.tokenize import word_tokenize
import os
import pandas as pd

class VoteClassifier(ClassifierI):
    def __init__(self,*classifiers):
        self._classifiers=classifiers
    
    def classify(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self,features):        
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        choice_votes=votes.count(mode(votes))
        confidence_level=(choice_votes/len(votes))
        return confidence_level

In [21]:
short_pos=open("C:/Users/HP/Downloads/positive.txt","r").read()
short_neg=open("C:/Users/HP/Downloads/negative.txt","r").read()

documents=[(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

all_words=[]
documents=[]
allowed_word_types=["J","R","V"]
#J is adject,r is adverb,v is verb
#---------------------------------------------LOGIC--------------------------------------------------------------
stop_words=set(stopwords.words('english'))
for p in short_pos.split("\n"):
    documents.append((p,"pos"))
    words=word_tokenize(p)
    words=[w for w in words if not w in stop_words]
     
    position=nltk.pos_tag(words) #position_tag
    for w in position:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg.split("\n"):
    documents.append((p,"neg"))
    words=word_tokenize(p)
    words=[w for w in words if not w in stop_words]
    position=nltk.pos_tag(words) #position_tag
    for w in position:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [22]:
#---------------------------------------------PICKLE(method 1)--------------------------------------------------------------
os.chdir("C:/Users/HP/.spyder-py3")

with open("pickled_documents.pickle","wb") as save_documents:
    pickle.dump(documents,save_documents)#(what do we want to dump,where to dump)
    save_documents.close()

In [23]:
k=pd.read_pickle("pickled_documents.pickle", compression='infer')
len_pos=0
len_neg=0
for i in k:
    if i[1]=='pos':
        len_pos+=1
    else:
        len_neg+=1
print(len_pos)
print(len_neg)

5332
5332


In [31]:
#---------------------------------------------PICKLE(method 2)--------------------------------------------------------------
all_words=nltk.FreqDist(all_words)
word_features=list(all_words.keys())[:5000]

save_word_features=open("pickled_word_features5k.pickle","wb")
pickle.dump(word_features,save_word_features)
save_word_features.close()

def find_features(document):
    words=word_tokenize(document)
    features={}
    for w in word_features:
        features[w]= (w in words)
    return features

#print((find_features(movie_reviews.words("neg/cv000_29416.txt"))))
featuresets=[(find_features(rev),category) for (rev,category) in documents]

random.shuffle(featuresets)

save_classifier=open("pickled_featuresets5k.pickle","wb")
pickle.dump(featuresets,save_classifier)
save_classifier.close()

training_set=featuresets[:9500]
testing_set=featuresets[9500:]

In [44]:
#CLASSIFIERS
Original_classifier=nltk.NaiveBayesClassifier.train(training_set) 
print("Original_Classifier",nltk.classify.accuracy(Original_classifier,testing_set)*100)
Original_classifier.show_most_informative_features(20)
save_classifier=open("pickled_originalnaivebayes5k.pickle","wb")   # will work without training again and again
pickle.dump(Original_classifier,save_classifier)
save_classifier.close()

SGDC_classifier=SklearnClassifier(SGDClassifier(max_iter=1000))
SGDC_classifier.train(training_set)
print("SGDC accuracy",nltk.classify.accuracy(SGDC_classifier,testing_set)*100)
save_classifier=open("pickled_SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier,save_classifier)
save_classifier.close()

Original_Classifier 69.67353951890034
Most Informative Features
              engrossing = True              pos : neg    =     21.3 : 1.0
                captures = True              pos : neg    =     15.9 : 1.0
               inventive = True              pos : neg    =     15.2 : 1.0
                    dull = True              neg : pos    =     14.1 : 1.0
              refreshing = True              pos : neg    =     13.9 : 1.0
                    warm = True              pos : neg    =     13.2 : 1.0
               wonderful = True              pos : neg    =     11.6 : 1.0
                  stupid = True              neg : pos    =     10.8 : 1.0
             mesmerizing = True              pos : neg    =     10.5 : 1.0
                   quiet = True              pos : neg    =      9.9 : 1.0
                provides = True              pos : neg    =      9.9 : 1.0
                     son = True              pos : neg    =      9.8 : 1.0
                 intense = True     

In [33]:
MNB_classifier=SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MultinomialNB accuracy",nltk.classify.accuracy(MNB_classifier,testing_set)*100)
save_classifier=open("pickled_MNBclassifier5k.pickle","wb")
pickle.dump(MNB_classifier,save_classifier)
save_classifier.close()


BernoulliNB_classifier=SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB accuracy",nltk.classify.accuracy(BernoulliNB_classifier,testing_set)*100)
save_classifier=open("pickled_Bernoulli_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier,save_classifier)
save_classifier.close()


MultinomialNB accuracy 70.27491408934708
BernoulliNB accuracy 68.98625429553265


In [34]:
LogisticRegression_classifier=SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression accuracy",nltk.classify.accuracy(LogisticRegression_classifier,testing_set)*100)
save_classifier=open("pickled_LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier,save_classifier)
save_classifier.close()


SVC_classifier=SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC accuracy",nltk.classify.accuracy(SVC_classifier,testing_set)*100)
save_classifier=open("pickled_SVC_classifier5k.pickle","wb")
pickle.dump(SVC_classifier,save_classifier)
save_classifier.close()

LogisticRegression accuracy 69.7594501718213
SVC accuracy 47.07903780068728


In [35]:
LinearSVC_classifier=SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC accuracy",nltk.classify.accuracy(LinearSVC_classifier,testing_set)*100)
save_classifier=open("pickled_LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier,save_classifier)
save_classifier.close()


NuSVC_classifier=SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC accuracy",nltk.classify.accuracy(NuSVC_classifier,testing_set)*100)
save_classifier=open("pickled_NuSVC_classifier5k.pickle","wb")
pickle.dump(NuSVC_classifier,save_classifier)
save_classifier.close()

LinearSVC accuracy 67.26804123711341
NuSVC accuracy 69.7594501718213


In [36]:
voted_classifier= VoteClassifier(Original_classifier,
                                 MNB_classifier,
                                 BernoulliNB_classifier,
                                 SVC_classifier,
                                 LogisticRegression_classifier,
                                 LinearSVC_classifier,
                                 NuSVC_classifier) 

print(nltk.classify.accuracy(voted_classifier,testing_set)*100)

70.01718213058419


In [38]:
def sentiment(text):
    feats=find_features(text)
    #conf=VoteClassifier.confidence
    return voted_classifier.classify(feats)

In [42]:
print("Classifier",voted_classifier.classify(testing_set[0][0]),"Confidence:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classifier",voted_classifier.classify(testing_set[2][0]),"Confidence:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classifier",voted_classifier.classify(testing_set[3][0]),"Confidence:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classifier",voted_classifier.classify(testing_set[5][0]),"Confidence:",voted_classifier.confidence(testing_set[5][0])*100)
print("Classifier",voted_classifier.classify(testing_set[6][0]),"Confidence:",voted_classifier.confidence(testing_set[6][0])*100)
print("Classifier",voted_classifier.classify(testing_set[7][0]),"Confidence:",voted_classifier.confidence(testing_set[7][0])*100)

Classifier pos Confidence: 85.71428571428571
Classifier neg Confidence: 100.0
Classifier neg Confidence: 100.0
Classifier neg Confidence: 100.0
Classifier neg Confidence: 71.42857142857143
Classifier neg Confidence: 100.0
